## Importing Libraries

In [3]:
import numpy as np
import pandas as pd
import math
import heapq
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from est_MI import est_MI_JVHW, est_MI_MLE
from sklearn import preprocessing
from graphviz import Graph

### Adding graphviz package to PATH

In [61]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

## Loading Dataset

In [4]:
df=pd.read_csv("alarm10k.csv")

In [5]:
df.head()

,HISTORY,CVP,PCWP,HYPOVOLEMIA,LVEDVOLUME,LVFAILURE,STROKEVOLUME,ERRLOWOUTPUT,HRBP,HREKG,...,MINVOLSET,VENTMACH,VENTTUBE,VENTLUNG,VENTALV,ARTCO2,CATECHOL,HR,CO,BP
0,False,NORMAL,NORMAL,False,NORMAL,False,NORMAL,False,HIGH,HIGH,...,NORMAL,ZERO,ZERO,LOW,HIGH,LOW,HIGH,HIGH,HIGH,HIGH
1,False,NORMAL,NORMAL,False,NORMAL,False,LOW,False,HIGH,HIGH,...,HIGH,HIGH,ZERO,ZERO,ZERO,HIGH,HIGH,HIGH,LOW,LOW
2,False,NORMAL,NORMAL,False,NORMAL,False,NORMAL,False,NORMAL,HIGH,...,HIGH,HIGH,HIGH,LOW,HIGH,LOW,NORMAL,LOW,LOW,NORMAL
3,False,NORMAL,NORMAL,False,NORMAL,False,NORMAL,False,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,HIGH
4,False,NORMAL,NORMAL,False,NORMAL,False,NORMAL,False,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,LOW,HIGH,HIGH,HIGH,HIGH,HIGH


In [6]:
df.shape

(10000, 37)

## Checking for NaNs

In [7]:
df.isnull().values.any()

False

### Labeling the columns for easier lookup

In [8]:
labels={}
count=0
for i in df:
    labels[count]=i
    count+=1

Opening the dataset as a file to make it easier for iterating through the rows

In [9]:
f = open("alarm10k.csv", "r")
joints = {}
marg = {}

feature_length = df.shape[1]
data_size = df.shape[0]

for i in range(feature_length):
    marg[i] = defaultdict(float)

    for j in range(i+1, feature_length):
        joints[(i, j)] = defaultdict(float)

count_aggr = 0

## Computing Probabilites

In [10]:
flag=0
for line in f:
    if(flag==0):
        flag=1
        continue
    n = line.strip().split(",")
    #print(n)
    count_aggr += 1
    for i in range(feature_length):
        marg[i][n[i]] += 1./data_size

        for j in range(i+1, feature_length):
            joints[(i,j)][(n[i], n[j])] += 1./data_size

## Calculating Mutual info using plugin estimator

In [11]:
def cal_mutual_info(joints, marg, feature_length):
  
    graph={}
    weights={}
    for i in range(feature_length):
        graph[str(i)]=[]
        for j in range(i+1, feature_length):
            graph[str(i)].append(j)
            I = 0
            for x_u, p_x_u in marg[i].items():
                for x_v, p_x_v in marg[j].items():
                    if (x_u, x_v) in joints[(i, j)]:
                        p_x_uv = joints[(i, j)][(x_u, x_v)]
                        I += p_x_uv * (math.log(p_x_uv, 2) - math.log(p_x_u, 2) - math.log(p_x_v, 2))
            weights[(i,j)]=-I
    return graph,weights

## Prims algorithm to build MST

In [36]:
def prim(graph,feature_length,weights):
    visited=[]
    nbr=[]
    dist=[]
    for i in range(feature_length):
        visited.append(False)
        nbr.append(-1)
        dist.append(42069)
    te=[]
    visited[0]=True
    for j in graph['0']:
        nbr[j]=0
        dist[j]=weights[(0,j)]
    for i in range(1,feature_length):
        distance={}
        for u in range(feature_length):
            if(visited[u]==False):
                distance[u]=dist[u]
        k=min(distance,key=distance.get)
        visited[k]=True
        te.append((k,nbr[k]))
        for v in graph[str(k)]:
            if(visited[v]==False):
                if(dist[v]>weights[(k,v)]):
                    dist[v]=weights[(k,v)]
                    nbr[v]=k
    return te,dist

In [13]:
graph,weights=cal_mutual_info(joints,marg,feature_length)

In [37]:
te,dist=prim(graph,feature_length,weights)

In [62]:
plotgraph(te,'MST_plugin')

## Graphs the tree and saves the figures.

In [59]:
def plotgraph(te,fname):
    count=0
    edges2=te.copy()
    for i in edges2:
        edges2[count]=list(edges2[count])
        edges2[count].append(labels[i[0]])
        edges2[count].append(labels[i[1]])
        edges2[count].pop(0)
        edges2[count].pop(0)
        count+=1
    g = Graph('G', filename=fname, engine='sfdp')
    for i in edges2:
        g.edge(i[0],i[1])
    g.view()

## Calcuting Mutual Info using JVHW estimator

In [16]:
def jvhw_mutual_info(df):
    le = preprocessing.LabelEncoder()
    for i in df:
        df[i]=le.fit_transform(df[i])
    jvhw={}
    graph={}
    for i in range(feature_length):
        graph[str(i)]=[]
        for j in range(i+1, feature_length):
            graph[str(i)].append(j)
            jvhw[(i,j)]=-est_MI_JVHW(df[labels[i]],df[labels[j]])
    return graph,jvhw

In [17]:
jvhw_graph,jvhw_info=jvhw_mutual_info(df)

In [38]:
jvhw_te,jvhw_dist=prim(jvhw_graph,feature_length,jvhw_info)

In [63]:
plotgraph(jvhw_te,'MST_JVHW')

## Comparing plugin estimator and JVWH estimator

In [56]:
def l1norm(jvhw,weights):
    l1norm=0
    length=len(jvhw)
    for i in range(length):
        if(i==0):
            continue
        l1norm+=np.abs(jvhw[i][0]-weights[i])
    print(l1norm)
    return l1norm

In [57]:
l1norm=l1norm(jvhw_dist,dist)

0.007700252060885924


The L1 norm between mutual information matrices are very low, which implies both are almost equal